# Extracción y limpieza de datos

## Lectura e importación de los Datasets sobre datos genómicos benignos y patogénicos

En este Notebook se procede a realizar una limpieza de los dos datasets con el fin de dejar los datos preparados para poder trabajar con ellos.
Para ello, se seguirán los siguientes pasos:

- Primero, importamos los dos datasets.
- Comprobamos qué columnas son numéricas inicialmente y cuáles no.
- Comprobamos qué columnas numéricas no aparecen como tal (debido a que contengan caracteres indicando el nulo mal codificados, aparezcan como un rango de valores,...) y las formateamos.
- Comprobamos el número de nulos de cada columna. De esta forma, en caso de que haya columnas con un alto porcentaje de nulos serán eliminadas. La eliminación se hará en ambos datasets aunque solo sea bajo en uno de ellos, ya que la información que nos dan las columnas de estos datos es su comparación.
- Comprobamos que las columnas no numéricas están codificadas con un rango de valores categóricos determinado, para eliminar signos que puedan ser ruido
- Comprobamos si las variables numéricas pueden expresarse mediante una variable binaria, indicando si es benigna o patógena; para ello, nos basaremos en los datos que tenemos de cada uno de los atributos.
- Comprobamos el rango de valores con el que trabajan las variables numéricas en los dos datasets; si es posible, se realizará una normalización a valores entre 0 y 1 para hacerlos más útiles.
- Exportaremos los datasets en formato .csv para utilizarlos en los siguientes pasos


In [1]:
import pandas as pd
import numpy as np

#### Para mostrar todas las columnas
pd.set_option('display.max_columns', None, "display.max_columns", None)

df1 = pd.read_csv(r'datasets/benign.out',low_memory=False,error_bad_lines=False,sep='\t')
df2 = pd.read_csv(r'datasets/pathogenic.out',low_memory=False,error_bad_lines=False,sep='\t')

### Lectura de las columnas que queremos utilizar en el Dataset

Las columnas que vamos a utilizar en el dataset se encuentran listadas en un fichero columnas.txt

In [2]:
f = open("files/columnas.txt", "r")
columns = f.read().split("\n")
    
df1 = df1[columns]
df2 = df2[columns]

In [3]:
df1.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10896 entries, 0 to 10895
Data columns (total 144 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   #chr                                                             10896 non-null  object 
 1   pos(1-based)                                                     10896 non-null  int64  
 2   ref                                                              10896 non-null  object 
 3   alt                                                              10896 non-null  object 
 4   aaref                                                            10896 non-null  object 
 5   aaalt                                                            10896 non-null  object 
 6   hg19_chr                                                         10896 non-null  object 
 7   hg19_pos(1-based)                      

In [4]:
df2.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5515 entries, 0 to 5514
Data columns (total 144 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   #chr                                                             5515 non-null   object 
 1   pos(1-based)                                                     5515 non-null   int64  
 2   ref                                                              5515 non-null   object 
 3   alt                                                              5515 non-null   object 
 4   aaref                                                            5515 non-null   object 
 5   aaalt                                                            5515 non-null   object 
 6   hg19_chr                                                         5515 non-null   object 
 7   hg19_pos(1-based)                        

Antes de comenzar, tenemos que ver qué significan las columnas para saber qué tipo de operaciones tenemos que aplicar sobre cada una de ellas.
Para ello, nos vamos a basar en la documentación de los siguientes enlaces:

- https://drive.google.com/file/d/1xck02Xa2f4y5RUf0Sb6GZQNqQScURy8B/view
- https://vatlab.github.io/vat-docs/applications/annotation/variants/dbnsfp/

El valor de las columnas se recogerá en un archivo adicional.

Primero, vamos a comprobar qué columnas están multivaluadas (contienen varios valores)

In [4]:
multival = []
for col in columns:
    for val in df1[col].unique():
        if ';' in str(val):
            multival.append(col)
            break
    for val in df2[col].unique():
        if ';' in str(val):
            if col not in multival:
                multival.append(col)
            break     
df1[multival]

,aapos,SIFT_score,SIFT4G_score,Polyphen2_HDIV_score,Polyphen2_HVAR_score,MutationTaster_score,MutationAssessor_score,FATHMM_score,PROVEAN_score,VEST4_score,MVP_score,MPC_score,DEOGEN2_score,LIST-S2_score,Aloft_prob_Tolerant,Aloft_prob_Recessive,Aloft_prob_Dominant
0,4,0.258,0.304,.,.,1,-1.1,-1.02,-0.27,0.071,0.553347326629,0.626244545349,.,0.29767,.,.,.
1,258;120,0.138;.,0.1;0.1,.;.,.;.,0.999476,2.43;.,-0.88;.,-2.99;.,0.25;0.234,0.834527163076;0.834527163076,0.872226974682;.,.;.,0.923708;0.927707,.;.,.;.,.;.
2,353;215,0.649;.,0.605;0.531,.;.,.;.,1,0.105;.,-0.84;.,-0.05;.,0.056;0.07,0.546866791787;0.546866791787,0.631211528713;.,.;.,0.147585;0.117888,.;.,.;.,.;.
3,375;237,0.022;.,0.418;0.417,.;.,.;.,1,1.58;.,-0.89;.,-1.15;.,0.455;0.463,0.710180199758;0.710180199758,0.942051258392;.,.;.,0.888711;0.891911,.;.,.;.,.;.
4,510;372,0.236;.,0.275;0.293,.;.,.;.,0.999792,0.325;.,3.85;.,-2.72;.,0.36;0.352,0.695986974221;0.695986974221,0.550584074293;.,.;.,0.756924;0.776522,.;.,.;.,.;.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10891,122;2257;168,0.081;0.252;.,0.361;0.966;.,.;0.245;.,.;0.155;.,0.995901;0.945543,.;0.485;.,-4.72;-4.72;.,-1.48;-1.19;.,0.052;0.014;.,.;.;.,.;0.849239045082;.,.;0.775965;.,0.689031;0.705629;0.689031,.;.;.,.;.;.,.;.;.
10892,1260,1.0,1.0,0.0,0.0,1,-1.585,-5.13,0.94,0.007,.,0.0443853036192,0.233462,0.148285,.,.,.
10893,998,0.05,0.578,0.008,0.002,1,1.79,-5.47,-1.7,0.647,0.820936177805,0.0469799550025,0.394953,0.29957,.,.,.
10894,503,0.031,0.038,0.992,0.594,0.999515,0.855,-6.04,-0.99,0.667,0.783423680238,1.47171924055,0.790767,0.688631,.,.,.


Mientras que muchas columnas son datos provenientes de modelos de los cuales hay que escoger el valor más alto (ya que es el que indica el valor más patógeno), no todas cumplen esta característica. Antes de hacer una modificación, vamos a ver cuales son:

 - FATHMM_score
 - PROVEAN_score
 - SIFT_score   
 - SIFT4G_score
 - Polyphen2_HDIV_score
 - Polyphen2_HVAR_score                            

En estas columnas, el significado de los valores está invertido, es decir: el mínimo es el valor más patógeno. Aplicaremos una elección del valor mínimo; en el resto, cogeremos el valor mayor de la colección de datos.

(ni aapos ni 'Aloft_prob_Tolerant', 'Aloft_prob_Recessive', 'Aloft_prob_Dominant' son scores, por lo que, por el momento, las dejaremos tal cual. Lo que si que haremos será sustituir los puntos por valores nulos, para comprobar si estas columnas contienen un número de datos aceptable)

### Conseguir los valores máximos y mínimos de celdas con valores múltiples

##### GetMaxOfRangeValues(values)

Dada una cadena (string) values con una serie de valores numéricos separados por ';' el formato 'a;b;c;...;n', devuelve el valor máximo de la lista de valores (ignora los puntos, los nulos, las variables numéricas únicas )

##### GetMinOfRangeValues(values)

Dada una cadena (string) values con una serie de valores numéricos separados por ';' el formato 'a;b;c;...;n', devuelve el valor mínimo de la lista de valores (ignora los puntos, los nulos, las variables numéricas únicas )

In [5]:
def GetMaxOfRangeValues(values):
    ### detectar si es un valor nulo
    if values != None:
        
        ### si es un único valor numérico lo devolvemos
        if type(values) == int or type(values) == float: return values
        else:
            
            ### Si la cadena contiene ; (colecciones) o '.' (posibles valores nulos) la tratamos
            if ";" in values or "." in values:
                try:

                    tmp = values.split(";")
                    tmp2 =[value for value in tmp if value != '.']

                    ### Si hay valores numéricos (que no sean colecciones solo de puntos) devolvemos el máximo
                    ### Si no, devolvemos un nulo
                    if len(tmp2) > 0:

                        return float(max(tmp2))
                    else:

                        return np.nan
                except:
                    print(values)
                    return values
                
            ### Si no, devolvemos la original
            else:
                
                return values
    else:
        
        return np.nan 

In [6]:
def GetMinOfRangeValues(values):
    ### detectar si es un valor nulo
    if values != None:
        
        ### si es un único valor numérico lo devolvemos
        if type(values) == int or type(values) == float: return values
        else:
            
            ### Si la cadena contiene ; (colecciones) o '.' (posibles valores nulos) la tratamos
            if ";" in values or "." in values:
                
                try:
                    
                    tmp = values.split(";")
                    tmp2 =[value for value in tmp if value != '.']

                    ### Si hay valores numéricos (que no sean colecciones solo de puntos) devolvemos el máximo
                    ### Si no, devolvemos un nulo
                    if len(tmp2) > 0:

                        return float(min(tmp2))

                    else:
                        return np.nan
                    
                except:
                    print(values)
                    return values
                
            ### Si no, devolvemos la original
            else:
                
                return values
    else:
        
        return np.nan 

In [7]:
reverse = ['FATHMM_score','PROVEAN_score','SIFT_score','SIFT4G_score','Polyphen2_HDIV_score','Polyphen2_HVAR_score']
for col in reverse:
    df1[col] = df1[col].apply(lambda x: GetMinOfRangeValues(x))
    df2[col] = df2[col].apply(lambda x: GetMinOfRangeValues(x))
    
to_remove = ['aapos','Aloft_prob_Tolerant','Aloft_prob_Recessive','Aloft_prob_Dominant']
not_reversed = [x for x in columns if x not in to_remove]
not_reversed = [x for x in not_reversed if x not in reverse]
for col in not_reversed:
    df1[col] = df1[col].apply(lambda x: GetMaxOfRangeValues(x))
    df2[col] = df2[col].apply(lambda x: GetMaxOfRangeValues(x))

In [8]:
def SetNanValues(x):
    if x != np.nan and x != None:
        if ';' in str(x) or '.' in str(x):
            tmp = str(x).split(";")
            tmp2 =[value for value in tmp if value != '.']
            if len(tmp2) > 0:
                return x
            else :
                return np.nan
        else:
            return np.nan
    else:
        return np.nan
    
for col in ['aapos','Aloft_prob_Tolerant', 'Aloft_prob_Recessive', 'Aloft_prob_Dominant']:
    df1[col] = df1[col].apply(lambda x: SetNanValues(x))
    df2[col] = df2[col].apply(lambda x: SetNanValues(x)) 

### Variables numéricas del dataset de 'benign' y valores

In [9]:
df1.select_dtypes(include=np.number).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10896 entries, 0 to 10895
Data columns (total 70 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   pos(1-based)                10896 non-null  int64  
 1   hg19_pos(1-based)           10896 non-null  int64  
 2   SIFT_score                  10545 non-null  float64
 3   SIFT4G_score                10437 non-null  float64
 4   Polyphen2_HDIV_score        9816 non-null   float64
 5   Polyphen2_HVAR_score        9816 non-null   float64
 6   LRT_score                   9625 non-null   float64
 7   LRT_Omega                   9625 non-null   float64
 8   FATHMM_score                10528 non-null  float64
 9   PROVEAN_score               10564 non-null  float64
 10  VEST4_score                 10836 non-null  float64
 11  MetaSVM_score               10809 non-null  float64
 12  MetaLR_score                10809 non-null  float64
 13  M-CAP_score                 280

In [10]:
df1.describe()

,pos(1-based),hg19_pos(1-based),SIFT_score,SIFT4G_score,Polyphen2_HDIV_score,Polyphen2_HVAR_score,LRT_score,LRT_Omega,FATHMM_score,PROVEAN_score,VEST4_score,MetaSVM_score,MetaLR_score,M-CAP_score,REVEL_score,MVP_score,MPC_score,PrimateAI_score,DEOGEN2_score,BayesDel_addAF_score,BayesDel_noAF_score,ClinPred_score,LIST-S2_score,CADD_raw_hg19,CADD_phred_hg19,DANN_score,fathmm-MKL_coding_score,fathmm-XF_coding_score,Eigen-raw_coding,Eigen-phred_coding,Eigen-PC-raw_coding,Eigen-PC-phred_coding,GenoCanyon_score,integrated_fitCons_score,GM12878_fitCons_score,H1-hESC_fitCons_score,HUVEC_fitCons_score,LINSIGHT,GERP++_RS,phyloP100way_vertebrate,phyloP30way_mammalian,phyloP17way_primate,phastCons100way_vertebrate,phastCons30way_mammalian,phastCons17way_primate,SiPhy_29way_logOdds,1000Gp3_AF,1000Gp3_AFR_AF,1000Gp3_EUR_AF,1000Gp3_AMR_AF,1000Gp3_EAS_AF,1000Gp3_SAS_AF,TWINSUK_AF,ALSPAC_AF,UK10K_AF,ESP6500_AA_AF,ESP6500_EA_AF,ExAC_AF,ExAC_nonTCGA_AF,ExAC_nonpsych_AF,gnomAD_exomes_AF,gnomAD_genomes_AF,gnomAD_genomes_POPMAX_AF,P(HI),HIPred_score,GHIS,P(rec),GDI,GDI-Phred,LoFtool_score
count,1.089600e+04,1.089600e+04,10545.000000,10437.000000,9816.000000,9816.000000,9625.000000,9625.000000,10528.000000,10564.000000,10836.000000,10809.000000,10809.000000,2809.000000,10809.000000,7790.000000,9681.000000,10561.000000,9969.000000,10886.000000,10886.000000,10837.000000,10513.000000,10896.000000,10896.000000,10896.000000,10896.000000,10246.000000,10315.000000,10315.000000,10315.000000,10315.000000,10896.000000,10367.000000,10367.000000,10367.000000,10367.000000,23.000000,10890.000000,10896.000000,10896.000000,10896.000000,10896.000000,10896.000000,10896.000000,10874.000000,10050.000000,10050.000000,10050.000000,10050.000000,10050.000000,10050.000000,5930.000000,5930.000000,5930.000000,9259.000000,9259.000000,10775.000000,10750.000000,10766.000000,10786.000000,10727.000000,10727.000000,2462.000000,2498.000000,2228.000000,2090.000000,2502.000000,2502.000000,2218.000000
mean,7.780955e+07,7.803485e+07,0.250889,0.284394,0.351610,0.247714,0.151180,8.121377,-0.415346,-1.341702,0.304747,-0.691645,0.193481,0.136305,0.225933,0.573413,0.375759,0.444461,0.249658,-0.427538,-0.368516,0.041661,0.697971,1.765902,16.682364,0.854079,0.540429,0.340044,-0.387152,2.312673,-0.350304,2.471486,0.772017,0.629029,0.622385,0.639234,0.624179,0.407182,2.319803,2.401601,0.412908,0.215375,0.606661,0.602073,0.600295,10.065975,0.067581,0.073991,0.064869,0.065695,0.065757,0.064917,0.106759,0.106877,0.106820,0.075279,0.067160,0.060810,0.060965,0.061176,0.061653,0.063109,0.061320,0.324520,0.499413,0.509718,0.292036,3940.656848,9.672830,0.479371
std,5.954830e+07,5.974846e+07,0.300925,0.301975,0.421211,0.358276,0.257095,226.519277,2.180888,1.807624,0.245622,0.515675,0.248972,0.202026,0.197819,0.271821,0.384867,0.170160,0.230272,0.220369,0.272038,0.104624,0.237580,1.344879,8.188233,0.221494,0.388482,0.300672,0.721707,2.535736,0.724608,2.680154,0.381573,0.102779,0.087718,0.088828,0.085339,0.376656,3.578101,2.816671,0.898194,0.711293,0.456973,0.426086,0.407798,5.173938,0.171184,0.177084,0.178458,0.175764,0.187365,0.177575,0.219548,0.219616,0.219568,0.176229,0.181523,0.168677,0.168784,0.168126,0.170672,0.167036,0.171954,0.258143,0.190784,0.073630,0.222770,8681.126403,7.692522,0.345802
min,8.875600e+04,1.387550e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-12.400000,-12.580000,0.002000,-2.005800,0.000000,0.000950,0.000000,0.013882,0.000382,0.154256,0.000139,-1.083610,-1.197980,0.000010,0.002400,-2.771251,0.001000,0.036852,0.000000,0.003400,-2.793459,0.000141,-2.869872,0.000139,0.000001,0.001892,0.000000,0.000000,0.000000,0.052183,-12.300000,-8.218000,-6.245000,-6.613000,0.000000,0.000000,0.000000,0.002200,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000132,0.000000,0.000000,0.000008,0.000009,0.000011,0.000004,0.000007,0.000007,0.000840,0.111908,0.373524,0.001020,1.967080,0.064200,0.000207
25%,3.233336e+07,3.254201e+07,0.017000,0.030000,0.001000,0.00200

### Variables numéricas del dataset de 'pathogenic' y valores

In [11]:
df2.select_dtypes(include=np.number).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5515 entries, 0 to 5514
Data columns (total 69 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   pos(1-based)                5515 non-null   int64  
 1   hg19_pos(1-based)           5515 non-null   int64  
 2   SIFT_score                  5327 non-null   float64
 3   SIFT4G_score                5364 non-null   float64
 4   Polyphen2_HDIV_score        5055 non-null   float64
 5   Polyphen2_HVAR_score        5055 non-null   float64
 6   LRT_score                   5219 non-null   float64
 7   LRT_Omega                   5219 non-null   float64
 8   FATHMM_score                5379 non-null   float64
 9   PROVEAN_score               5378 non-null   float64
 10  MetaSVM_score               5447 non-null   float64
 11  MetaLR_score                5447 non-null   float64
 12  M-CAP_score                 5430 non-null   float64
 13  REVEL_score                 5447 

In [12]:
df2.describe()

,pos(1-based),hg19_pos(1-based),SIFT_score,SIFT4G_score,Polyphen2_HDIV_score,Polyphen2_HVAR_score,LRT_score,LRT_Omega,FATHMM_score,PROVEAN_score,MetaSVM_score,MetaLR_score,M-CAP_score,REVEL_score,MVP_score,MPC_score,PrimateAI_score,DEOGEN2_score,BayesDel_addAF_score,BayesDel_noAF_score,ClinPred_score,LIST-S2_score,CADD_raw_hg19,CADD_phred_hg19,DANN_score,fathmm-MKL_coding_score,fathmm-XF_coding_score,Eigen-raw_coding,Eigen-phred_coding,Eigen-PC-raw_coding,Eigen-PC-phred_coding,GenoCanyon_score,integrated_fitCons_score,GM12878_fitCons_score,H1-hESC_fitCons_score,HUVEC_fitCons_score,LINSIGHT,GERP++_RS,phyloP100way_vertebrate,phyloP30way_mammalian,phyloP17way_primate,phastCons100way_vertebrate,phastCons30way_mammalian,phastCons17way_primate,SiPhy_29way_logOdds,1000Gp3_AF,1000Gp3_AFR_AF,1000Gp3_EUR_AF,1000Gp3_AMR_AF,1000Gp3_EAS_AF,1000Gp3_SAS_AF,TWINSUK_AF,ALSPAC_AF,UK10K_AF,ESP6500_AA_AF,ESP6500_EA_AF,ExAC_AF,ExAC_nonTCGA_AF,ExAC_nonpsych_AF,gnomAD_exomes_AF,gnomAD_genomes_AF,gnomAD_genomes_POPMAX_AF,P(HI),HIPred_score,GHIS,P(rec),GDI,GDI-Phred,LoFtool_score
count,5.515000e+03,5.515000e+03,5327.000000,5364.000000,5055.000000,5055.000000,5219.000000,5219.000000,5379.000000,5378.000000,5447.000000,5447.000000,5430.000000,5447.000000,5463.000000,5023.000000,5243.000000,5177.000000,5515.000000,5515.000000,5487.000000,5333.000000,5515.000000,5515.000000,5515.000000,5515.000000,5081.000000,5165.000000,5165.000000,5165.000000,5165.000000,5515.000000,5179.000000,5179.000000,5179.000000,5179.000000,20.000000,5515.000000,5515.000000,5515.000000,5515.000000,5515.000000,5515.000000,5515.000000,5515.000000,405.000000,405.000000,405.000000,405.000000,405.000000,405.000000,413.000000,413.000000,413.000000,771.000000,771.000000,2107.000000,1970.000000,1972.000000,2493.000000,2179.000000,2179.000000,1071.000000,1057.000000,995.000000,970.000000,1066.000000,1066.000000,966.000000
mean,7.190392e+07,7.218650e+07,0.021593,0.032175,0.893316,0.827701,0.010699,0.060644,-3.525490,-4.869182,0.712253,0.809497,0.555218,0.822627,0.947211,1.148306,0.721079,0.791051,0.385344,0.395270,0.910965,0.930674,3.807320,27.156349,0.990399,0.928688,0.822104,0.664201,8.502959,0.603396,8.472333,0.941553,0.642712,0.630761,0.649706,0.637083,0.897749,4.707782,6.475314,1.007663,0.581716,0.964042,0.867314,0.820749,15.375034,0.003771,0.003874,0.004813,0.004374,0.002385,0.003558,0.004582,0.004688,0.004636,0.001969,0.002516,0.000806,0.000854,0.000836,0.000719,0.000852,0.001034,0.422047,0.600293,0.540527,0.417126,1162.701048,4.994870,0.163520
std,5.634212e+07,5.627303e+07,0.091150,0.105586,0.258170,0.300346,0.077640,0.181625,2.281173,2.535656,0.529573,0.230230,0.291229,0.172093,0.090037,0.815311,0.156388,0.231637,0.183825,0.218021,0.204748,0.104249,0.829110,3.910514,0.045217,0.138139,0.208275,0.367713,4.387142,0.349912,4.859548,0.209383,0.099387,0.088966,0.092004,0.087353,0.090374,1.533089,2.760329,0.403253,0.262563,0.165461,0.269690,0.293776,3.550790,0.024247,0.028942,0.031450,0.031624,0.015613,0.028948,0.032182,0.032159,0.032160,0.018705,0.022732,0.012103,0.012452,0.012171,0.011771,0.012437,0.012527,0.281691,0.189356,0.082970,0.302922,2082.322879,4.713407,0.235468
min,1.770120e+05,2.184710e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-12.400000,-13.960000,-1.216200,0.000000,0.004008,0.011000,0.067524,0.000372,0.178245,0.002016,-0.618487,-0.720656,0.000220,0.042096,-1.016770,0.002000,0.132257,0.002060,0.008185,-1.833750,0.020124,-1.913915,0.020338,0.000002,0.006267,0.000000,0.000000,0.055017,0.694796,-12.200000,-3.591000,-4.103000,-2.973000,0.000000,0.000000,0.000000,0.555100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000132,0.000000,0.000000,0.000008,0.000009,0.000011,0.000004,0.000007,0.000007,0.030370,0.111908,0.373524,0.068460,1.235830,0.042850,0.000482
25%,3.122740e+07,2.962108e+07,0.000000,0.000000,0.981000,0.828000,0.000000,0.000000,-5.130000,-6.647500,0.564350,0.728750,0.296715,0.755000,0.941616,0.506211,0.619717,0.711488,0.28462

## Reconvertimos a numéricas aquellas columnas en las que las conversiones previas fallaron

In [13]:
result = []

def ConvertToNumericValue(col,value):
    if value != None:
        ### si es un valor numérico lo devolvemos
        if type(value) == int or type(value) == float: 
            return value
        ### si no
        else:
            ### comprobamos si es un valor nulo expresado con un punto
            if "." == str(value):
                return np.nan
            ### Si no, intentamos hacer casting a float
            else:
                try:
                    ### si funciona, devolvemos el valor
                    return float(value)
                ### si no, devolvemos el elemento original
                except:
                    if col not in result:
                        result.append(col)
                    if value not in result:
                        result.append(value)
                    return value
    ### si el elemento es nulo
    else:
        return np.nan     

Con la impresión de los valores, comprobamos si existen símbolos o patrones en los datos incorrectos.

In [14]:
for col in columns:
    df1[col] = df1[col].apply(lambda x: ConvertToNumericValue(col,x))
    df2[col] = df2[col].apply(lambda x: ConvertToNumericValue(col,x))    
print(result)    

['#chr', 'X', 'ref', 'G', 'C', 'A', 'T', 'alt', 'aaref', 'R', 'Q', 'V', 'E', 'L', 'P', 'S', 'H', 'D', 'M', 'F', 'I', 'W', 'N', 'Y', 'K', 'aaalt', 'hg19_chr', 'aapos', '258;120', '353;215', '375;237', '510;372', '554;416', '728;590', '852;714', '1026;888', '1088;950', '1135;997', '1289;1151', '1322;1184', '1429;1291', '1451;1313', '1514;1376', '1565;1427', '1666;1528', '1883;1749', '1909;1794', '2025;1910', '174;174', '220;248;220;220;200;46', '294;274;274', '249;229;229', '145;145;145;1', '111;111;111', '34;34;34;34', '477;476;477;476;285', '523;522;523;522;331', '534;533;534;533;342', '563;562;563;562;371', '571;570;571;570;379', '634;633;634;633;442', '765;764;765;764;573', '819;818;819;818;627', '824;823;824;823;632', '837;836;837;836;645', '1045;1044;1044;1044;852', '1102;1101;1101;1101;909', '765;764', '753;752', '740;739', '656;655', '618;617', '569;568', '544;543', '541;540', '497;497', '315;315', '171;171', '29;29', '84;84;84', '236;236;236', '357;357;357;142', '368;368;368;153

 - aapos está sin formatear
 - MutPred_score contiene un guión entre sus símbolos que hay que convertir a nulo.

In [15]:
def MutPred_score_Format(value):
    if value != None:
        ### si es un valor numérico lo devolvemos
        if type(value) == int or type(value) == float: 
            return value
        ### si no
        else:
            ### comprobamos si es un valor nulo expresado con un punto
            if "." == str(value) or '-' == str(value):
                return np.nan
            ### Si no, intentamos hacer casting a float
            else:
                try:
                    ### si funciona, devolvemos el valor
                    return float(value)
                ### si no, devolvemos el elemento original
                except:
                    return value
    ### si el elemento es nulo
    else:
        return np.nan     
    
df1['MutPred_score'] = df1['MutPred_score'].apply(lambda x: MutPred_score_Format(x))
df2['MutPred_score'] = df2['MutPred_score'].apply(lambda x: MutPred_score_Format(x))   

Para 'aapos', una opción interesante es dividir su valor en tantas filas como valores tenga, modificando únicamente ese parámetro.

In [16]:
def toList(x):
    if type(x)==str: 
        return x.split(";") 
    else: 
        return x

df1['aapos'] = df1['aapos'].apply(lambda x: toList(x))
df1 = df1.apply( pd.Series.explode )

df2['aapos'] = df2['aapos'].apply(lambda x: toList(x))
df2 = df2.apply( pd.Series.explode )

### Comprobamos los valores distintos de los datos no numéricos

In [17]:
df1.select_dtypes(exclude=np.number).info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39229 entries, 0 to 10895
Data columns (total 21 columns):
 #   Column                                                           Non-Null Count  Dtype 
---  ------                                                           --------------  ----- 
 0   #chr                                                             39229 non-null  object
 1   ref                                                              39229 non-null  object
 2   alt                                                              39229 non-null  object
 3   aaref                                                            39225 non-null  object
 4   aaalt                                                            39225 non-null  object
 5   hg19_chr                                                         39229 non-null  object
 6   aapos                                                            36675 non-null  object
 7   Aloft_prob_Tolerant                              

In [18]:
df2.select_dtypes(exclude=np.number).info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22834 entries, 0 to 5514
Data columns (total 21 columns):
 #   Column                                                           Non-Null Count  Dtype 
---  ------                                                           --------------  ----- 
 0   #chr                                                             22834 non-null  object
 1   ref                                                              22834 non-null  object
 2   alt                                                              22834 non-null  object
 3   aaref                                                            22796 non-null  object
 4   aaalt                                                            22796 non-null  object
 5   hg19_chr                                                         22834 non-null  object
 6   aapos                                                            21733 non-null  object
 7   Aloft_prob_Tolerant                               

### Comprobamos los valores distintos de los scores

In [19]:
result = [x for x in columns if 'score' in x]
df1[result].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39229 entries, 0 to 10895
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   SIFT_score                38461 non-null  float64
 1   SIFT4G_score              37568 non-null  float64
 2   Polyphen2_HDIV_score      36266 non-null  float64
 3   Polyphen2_HVAR_score      36266 non-null  float64
 4   LRT_score                 34713 non-null  float64
 5   MutationTaster_score      39070 non-null  float64
 6   MutationAssessor_score    34539 non-null  float64
 7   FATHMM_score              38326 non-null  float64
 8   PROVEAN_score             38500 non-null  float64
 9   VEST4_score               39112 non-null  float64
 10  MetaSVM_score             39079 non-null  float64
 11  MetaLR_score              39079 non-null  float64
 12  M-CAP_score               12486 non-null  float64
 13  REVEL_score               39079 non-null  float64
 14  MutPre

MutPred_score apenas tiene valores; M-CAP_score, HIPred_score y LoFtool_score no tienen una cantidad muy alta de datos

In [20]:
df2[result].info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22834 entries, 0 to 5514
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   SIFT_score                22282 non-null  float64
 1   SIFT4G_score              22371 non-null  float64
 2   Polyphen2_HDIV_score      21676 non-null  float64
 3   Polyphen2_HVAR_score      21676 non-null  float64
 4   LRT_score                 21998 non-null  float64
 5   MutationTaster_score      22777 non-null  float64
 6   MutationAssessor_score    20706 non-null  float64
 7   FATHMM_score              22375 non-null  float64
 8   PROVEAN_score             22404 non-null  float64
 9   VEST4_score               22577 non-null  float64
 10  MetaSVM_score             22652 non-null  float64
 11  MetaLR_score              22652 non-null  float64
 12  M-CAP_score               22531 non-null  float64
 13  REVEL_score               22652 non-null  float64
 14  MutPred

In [21]:
matching = [s for s in columns if "Gene damage prediction" in s]
for col in matching:
    print(df1[col].unique())
    print(df2[col].unique())

['Medium' nan 'High' 'Low']
[nan 'Medium' 'High' 'Low']
['Medium' nan 'High' 'Low']
[nan 'Medium' 'High' 'Low']
['Medium' nan 'High' 'Low']
[nan 'Medium' 'Low' 'High']
['Medium' nan 'High' 'Low']
[nan 'Medium' 'Low' 'High']
['Medium' nan 'High']
[nan 'Medium' 'High' 'Low']
['Medium' nan 'High']
[nan 'Medium' 'High' 'Low']
['Medium' nan 'High' 'Low']
[nan 'Medium' 'Low' 'High']
['Medium' nan 'High' 'Low']
[nan 'Medium' 'High' 'Low']
['Medium' nan 'High' 'Low']
[nan 'Medium' 'High' 'Low']
['Medium' nan 'High' 'Low']
[nan 'Medium' 'High' 'Low']


Las columnas Gene damage prediction están bien formateadas al mismo tipo de valores: 'Low', 'Medium' y 'High'.

In [22]:
matching = [s for s in columns if "ref" == s or "alt" == s]
for col in matching:
    print(df1[col].unique())
    print(df2[col].unique())

['G' 'C' 'A' 'T']
['C' 'G' 'T' 'A']
['C' 'T' 'G' 'A']
['T' 'A' 'C' 'G']


In [23]:
matching = [s for s in columns if "aaref" == s or "aaalt" == s]
for col in matching:
    print(df1[col].unique())
    print(df2[col].unique())

['R' 'T' 'Q' 'A' 'G' 'V' 'E' 'L' 'P' 'S' 'H' 'D' 'M' 'F' 'I' 'W' 'N' 'Y'
 'C' 'K' nan 'X']
['R' 'P' 'M' 'I' 'K' 'L' 'G' 'H' 'E' 'N' 'F' 'T' 'A' 'V' 'S' 'W' 'C' 'D'
 'Y' 'Q' nan 'X']
['P' 'I' 'R' 'S' 'M' 'V' 'N' 'F' 'L' 'W' 'C' 'T' 'H' 'K' 'D' 'G' 'E' 'A'
 'Q' 'Y' nan 'X']
['W' 'L' 'V' 'T' 'R' 'S' 'C' 'E' 'K' 'Q' 'P' 'G' 'M' 'Y' 'F' 'H' 'I' 'A'
 'D' 'N' 'X' nan]


Las columnas 'ref' y 'alt' también están bien formateadas (Valores adenina (A), citosina (C), guanina (G) y timina (T)).

Las columnas 'aaref' y 'aaalt' también están igualmente codificadas con las siguientes posibilidades:
['R' 'T' 'Q' 'A' 'G' 'V' 'E' 'L' 'P' 'S' 'H' 'D' 'M' 'F' 'I' 'W' 'N' 'Y' 'C' 'K' 'X'].

Vamos a comprobar el número de valores existentes para aapos y hg19_chr

In [24]:
print(len(df1['aapos'].unique()))
print(len(df2['aapos'].unique()))
print(len(df1['hg19_chr'].unique()))
print(len(df2['hg19_chr'].unique()))

4132
1729
23
23


In [25]:
print(df1['HIPred'].unique())
print(df2['HIPred'].unique())

['Y' nan 'N']
[nan 'Y' 'N']


presupondremos que 'Y' significa Si y 'N' significa No.

## Sustituímos en las columnas AF los nulos por el valor 0

In [26]:
result = [x for x in columns if x.endswith('_AF')]
for col in result:
    df1[col] = df1[col].fillna(0)
    df2[col] = df2[col].fillna(0)

## Comprobamos el porcentaje de nulos de cada columna

### Dataset 'Benign'

In [27]:
for col in columns:   
    print(col + " -> " + str(df1[col].isnull().sum()) + "/" + str(len(df1.index))+" (" + str(round((df1[col].isnull().sum()/len(df1.index)) * 100,2)) + ")%")

#chr -> 0/39229 (0.0)%
pos(1-based) -> 0/39229 (0.0)%
ref -> 0/39229 (0.0)%
alt -> 0/39229 (0.0)%
aaref -> 4/39229 (0.01)%
aaalt -> 4/39229 (0.01)%
hg19_chr -> 0/39229 (0.0)%
hg19_pos(1-based) -> 0/39229 (0.0)%
aapos -> 2554/39229 (6.51)%
SIFT_score -> 768/39229 (1.96)%
SIFT4G_score -> 1661/39229 (4.23)%
Polyphen2_HDIV_score -> 2963/39229 (7.55)%
Polyphen2_HVAR_score -> 2963/39229 (7.55)%
LRT_score -> 4516/39229 (11.51)%
LRT_Omega -> 4516/39229 (11.51)%
MutationTaster_score -> 159/39229 (0.41)%
MutationAssessor_score -> 4690/39229 (11.96)%
FATHMM_score -> 903/39229 (2.3)%
PROVEAN_score -> 729/39229 (1.86)%
VEST4_score -> 117/39229 (0.3)%
MetaSVM_score -> 150/39229 (0.38)%
MetaLR_score -> 150/39229 (0.38)%
M-CAP_score -> 26743/39229 (68.17)%
REVEL_score -> 150/39229 (0.38)%
MutPred_score -> 36213/39229 (92.31)%
MVP_score -> 10298/39229 (26.25)%
MPC_score -> 4670/39229 (11.9)%
PrimateAI_score -> 798/39229 (2.03)%
DEOGEN2_score -> 2841/39229 (7.24)%
BayesDel_addAF_score -> 17/39229 (0.04)

En los datos del dataset 'Benign', tenemos que las columnas que tienen un mayor número de nulos son:

- (aproximadamente 45%):

    - TWINSUK_AF
    - ALSPAC_AF 
    - UK10K_AF
    
    
- ( > 90 %):

    - Columnas relacionadas con el term. 'Interactions'
    - Columnas relacionadas con el term. 'P(HI)'
    - Columnas relacionadas con el term. 'HIPred'
    - Columnas relacionadas con el term. 'GHIS'
    - Columnas relacionadas con el term. 'P(rec)'
    - Columnas relacionadas con el term. 'GDI'
    - Aloft_prob_Tolerant -> 10894/10896 (99.98)%
    - Aloft_prob_Recessive -> 10894/10896 (99.98)%
    - Aloft_prob_Dominant -> 10894/10896 (99.98)%
    - Todas las predicciones de 'Gene damage prediction' 
    - LINSIGHT (99.86%)

Estas últimas directamente se descartan, ya que no podemos extraer nada de información de ellas.

### Dataset Pathogenic

In [28]:
for col in columns:   
    print(col + " -> " + str(df2[col].isnull().sum()) + "/"+str(len(df2.index))+" (" + str(round((df2[col].isnull().sum()/len(df2.index)) * 100,2)) + ")%")

#chr -> 0/22834 (0.0)%
pos(1-based) -> 0/22834 (0.0)%
ref -> 0/22834 (0.0)%
alt -> 0/22834 (0.0)%
aaref -> 38/22834 (0.17)%
aaalt -> 38/22834 (0.17)%
hg19_chr -> 0/22834 (0.0)%
hg19_pos(1-based) -> 0/22834 (0.0)%
aapos -> 1101/22834 (4.82)%
SIFT_score -> 552/22834 (2.42)%
SIFT4G_score -> 463/22834 (2.03)%
Polyphen2_HDIV_score -> 1158/22834 (5.07)%
Polyphen2_HVAR_score -> 1158/22834 (5.07)%
LRT_score -> 836/22834 (3.66)%
LRT_Omega -> 836/22834 (3.66)%
MutationTaster_score -> 57/22834 (0.25)%
MutationAssessor_score -> 2128/22834 (9.32)%
FATHMM_score -> 459/22834 (2.01)%
PROVEAN_score -> 430/22834 (1.88)%
VEST4_score -> 257/22834 (1.13)%
MetaSVM_score -> 182/22834 (0.8)%
MetaLR_score -> 182/22834 (0.8)%
M-CAP_score -> 303/22834 (1.33)%
REVEL_score -> 182/22834 (0.8)%
MutPred_score -> 5320/22834 (23.3)%
MVP_score -> 207/22834 (0.91)%
MPC_score -> 2115/22834 (9.26)%
PrimateAI_score -> 1064/22834 (4.66)%
DEOGEN2_score -> 579/22834 (2.54)%
BayesDel_addAF_score -> 0/22834 (0.0)%
BayesDel_noAF_

En los datos del dataset 'Pathogenic', tenemos que las columnas que tienen un mayor número de nulos son:

- ( > 90 %):

    - Columnas relacionadas con el term. 'Interactions'
    - Columnas relacionadas con el term. 'P(HI)'
    - Columnas relacionadas con el term. 'HIPred'
    - Columnas relacionadas con el term. 'GHIS'
    - Columnas relacionadas con el term. 'P(rec)'
    - Columnas relacionadas con el term. 'GDI'
    - Aloft_prob_Tolerant -> 10894/10896 (99.98)%
    - Aloft_prob_Recessive -> 10894/10896 (99.98)%
    - Aloft_prob_Dominant -> 10894/10896 (99.98)%
    - Todas las predicciones de 'Gene damage prediction' 
    - LoFtool_score (92.06)%

In [29]:
columns_to_remove = [x for x in columns if round((df2[x].isnull().sum()/len(df2.index)) * 100,2) > 85]
try:
    df1 = df1.drop(columns=columns_to_remove)
    df2 = df2.drop(columns=columns_to_remove)
    columns = [x for x in columns if x not in columns_to_remove]
except:
    # En caso de que ya hayan sido eliminadas
    pass

Vemos que el número de elementos en cada dataset es razonable excepto para la columna de  LINSIGHT, que apenas tiene valores; esta columna no nos aportará nada, por lo que tendremos que eliminarla también.

In [30]:
try:
    columns.remove("LINSIGHT")
    df1.pop("LINSIGHT")
    df2.pop("LINSIGHT")
    print("LINSIGHT Eliminado en ambos datasets")
except:
    # En caso de que ya hayan sido eliminadas
    pass

### Eliminamos los duplicados en ambos datasets (en caso de que haya)


In [31]:
df1 = df1.drop_duplicates()
df2 = df2.drop_duplicates()

In [32]:
df1.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22631 entries, 0 to 10895
Data columns (total 119 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   #chr                              object 
 1   pos(1-based)                      int64  
 2   ref                               object 
 3   alt                               object 
 4   aaref                             object 
 5   aaalt                             object 
 6   hg19_chr                          object 
 7   hg19_pos(1-based)                 int64  
 8   aapos                             object 
 9   SIFT_score                        float64
 10  SIFT4G_score                      float64
 11  Polyphen2_HDIV_score              float64
 12  Polyphen2_HVAR_score              float64
 13  LRT_score                         float64
 14  LRT_Omega                         float64
 15  MutationTaster_score              float64
 16  MutationAssessor_score            float

In [33]:
df2.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12167 entries, 0 to 5514
Data columns (total 119 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   #chr                              object 
 1   pos(1-based)                      int64  
 2   ref                               object 
 3   alt                               object 
 4   aaref                             object 
 5   aaalt                             object 
 6   hg19_chr                          object 
 7   hg19_pos(1-based)                 int64  
 8   aapos                             object 
 9   SIFT_score                        float64
 10  SIFT4G_score                      float64
 11  Polyphen2_HDIV_score              float64
 12  Polyphen2_HVAR_score              float64
 13  LRT_score                         float64
 14  LRT_Omega                         float64
 15  MutationTaster_score              float64
 16  MutationAssessor_score            float6

## Comparación del rango de valores entre los dos datasets

In [34]:
def GetColumnsWithMinAndMaxValuesInRange(df, a=0, b=1):
    tmp = df.describe().loc[['min','max']].T
    tmp = tmp[tmp['min'] >= a]
    tmp = tmp[tmp['max'] <= b]
    tmp = tmp.T
    return tmp

def GetColumnsWithMinAndMaxValuesOutOfRange(df, a=0, b=1):
    tmp = df.describe().loc[['min','max']].T
    tmp = tmp[tmp['min'] < a]
    tmp = tmp[tmp['max'] > b]
    tmp = tmp.T
    return tmp

In [35]:
GetColumnsWithMinAndMaxValuesInRange(df1,0,1)

,SIFT_score,SIFT4G_score,Polyphen2_HDIV_score,Polyphen2_HVAR_score,LRT_score,MutationTaster_score,VEST4_score,MetaLR_score,M-CAP_score,REVEL_score,MutPred_score,MVP_score,PrimateAI_score,DEOGEN2_score,ClinPred_score,LIST-S2_score,DANN_score,fathmm-MKL_coding_score,fathmm-XF_coding_score,GenoCanyon_score,integrated_fitCons_score,GM12878_fitCons_score,H1-hESC_fitCons_score,HUVEC_fitCons_score,phastCons100way_vertebrate,phastCons30way_mammalian,phastCons17way_primate,1000Gp3_AF,1000Gp3_AFR_AF,1000Gp3_EUR_AF,1000Gp3_AMR_AF,1000Gp3_EAS_AF,1000Gp3_SAS_AF,TWINSUK_AF,ALSPAC_AF,UK10K_AF,ESP6500_AA_AF,ESP6500_EA_AF,ExAC_AF,ExAC_Adj_AF,ExAC_AFR_AF,ExAC_AMR_AF,ExAC_EAS_AF,ExAC_FIN_AF,ExAC_NFE_AF,ExAC_SAS_AF,ExAC_nonTCGA_AF,ExAC_nonTCGA_Adj_AF,ExAC_nonTCGA_AFR_AF,ExAC_nonTCGA_AMR_AF,ExAC_nonTCGA_EAS_AF,ExAC_nonTCGA_FIN_AF,ExAC_nonTCGA_NFE_AF,ExAC_nonTCGA_SAS_AF,ExAC_nonpsych_AF,ExAC_nonpsych_Adj_AF,ExAC_nonpsych_AFR_AF,ExAC_nonpsych_AMR_AF,ExAC_nonpsych_EAS_AF,ExAC_nonpsych_FIN_AF,ExAC_nonpsych_NFE_AF,ExAC_nonpsych_SAS_AF,gnomAD_exomes_AF,gnomAD_exomes_AFR_AF,gnomAD_exomes_AMR_AF,gnomAD_exomes_ASJ_AF,gnomAD_exomes_EAS_AF,gnomAD_exomes_FIN_AF,gnomAD_exomes_NFE_AF,gnomAD_exomes_SAS_AF,gnomAD_exomes_POPMAX_AF,gnomAD_exomes_controls_AF,gnomAD_exomes_controls_AFR_AF,gnomAD_exomes_controls_AMR_AF,gnomAD_exomes_controls_ASJ_AF,gnomAD_exomes_controls_EAS_AF,gnomAD_exomes_controls_FIN_AF,gnomAD_exomes_controls_NFE_AF,gnomAD_exomes_controls_SAS_AF,gnomAD_exomes_controls_POPMAX_AF,gnomAD_genomes_AF,gnomAD_genomes_AFR_AF,gnomAD_genomes_AMR_AF,gnomAD_genomes_ASJ_AF,gnomAD_genomes_EAS_AF,gnomAD_genomes_FIN_AF,gnomAD_genomes_NFE_AF,gnomAD_genomes_AMI_AF,gnomAD_genomes_SAS_AF,gnomAD_genomes_POPMAX_AF
min,0.0,0.0,0.0,0.0,0.0,1.000000e-37,0.002,0.0000,0.000950,0.000,0.049,0.013882,0.154256,0.000139,0.000010,0.002400,0.036852,0.00000,0.003400,0.000001,0.001892,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,1.0,1.0,1.0,1.0,1.0,1.000000e+00,0.997,0.9999,0.994657,0.992,1.000,1.000000,0.973357,0.988043,0.999961,0.999865,0.999630,0.99907,0.987401,1.000000,0.839682,0.958517,0.858454,0.836244,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.999965,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [36]:
GetColumnsWithMinAndMaxValuesInRange(df2,0,1)

,SIFT_score,SIFT4G_score,Polyphen2_HDIV_score,Polyphen2_HVAR_score,LRT_score,MutationTaster_score,VEST4_score,MetaLR_score,M-CAP_score,REVEL_score,MutPred_score,MVP_score,PrimateAI_score,DEOGEN2_score,ClinPred_score,LIST-S2_score,DANN_score,fathmm-MKL_coding_score,fathmm-XF_coding_score,GenoCanyon_score,integrated_fitCons_score,GM12878_fitCons_score,H1-hESC_fitCons_score,HUVEC_fitCons_score,phastCons100way_vertebrate,phastCons30way_mammalian,phastCons17way_primate,1000Gp3_AF,1000Gp3_AFR_AF,1000Gp3_EUR_AF,1000Gp3_AMR_AF,1000Gp3_EAS_AF,1000Gp3_SAS_AF,TWINSUK_AF,ALSPAC_AF,UK10K_AF,ESP6500_AA_AF,ESP6500_EA_AF,ExAC_AF,ExAC_Adj_AF,ExAC_AFR_AF,ExAC_AMR_AF,ExAC_EAS_AF,ExAC_FIN_AF,ExAC_NFE_AF,ExAC_SAS_AF,ExAC_nonTCGA_AF,ExAC_nonTCGA_Adj_AF,ExAC_nonTCGA_AFR_AF,ExAC_nonTCGA_AMR_AF,ExAC_nonTCGA_EAS_AF,ExAC_nonTCGA_FIN_AF,ExAC_nonTCGA_NFE_AF,ExAC_nonTCGA_SAS_AF,ExAC_nonpsych_AF,ExAC_nonpsych_Adj_AF,ExAC_nonpsych_AFR_AF,ExAC_nonpsych_AMR_AF,ExAC_nonpsych_EAS_AF,ExAC_nonpsych_FIN_AF,ExAC_nonpsych_NFE_AF,ExAC_nonpsych_SAS_AF,gnomAD_exomes_AF,gnomAD_exomes_AFR_AF,gnomAD_exomes_AMR_AF,gnomAD_exomes_ASJ_AF,gnomAD_exomes_EAS_AF,gnomAD_exomes_FIN_AF,gnomAD_exomes_NFE_AF,gnomAD_exomes_SAS_AF,gnomAD_exomes_POPMAX_AF,gnomAD_exomes_controls_AF,gnomAD_exomes_controls_AFR_AF,gnomAD_exomes_controls_AMR_AF,gnomAD_exomes_controls_ASJ_AF,gnomAD_exomes_controls_EAS_AF,gnomAD_exomes_controls_FIN_AF,gnomAD_exomes_controls_NFE_AF,gnomAD_exomes_controls_SAS_AF,gnomAD_exomes_controls_POPMAX_AF,gnomAD_genomes_AF,gnomAD_genomes_AFR_AF,gnomAD_genomes_AMR_AF,gnomAD_genomes_ASJ_AF,gnomAD_genomes_EAS_AF,gnomAD_genomes_FIN_AF,gnomAD_genomes_NFE_AF,gnomAD_genomes_AMI_AF,gnomAD_genomes_SAS_AF,gnomAD_genomes_POPMAX_AF
min,0.0,0.0,0.0,0.0,0.000000,1.303790e-19,0.015,0.0000,0.004008,0.011,0.096,0.067524,0.178245,0.002016,0.000220,0.042096,0.132257,0.00206,0.008185,0.000002,0.006267,0.000000,0.000000,0.055017,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.0,1.0,1.0,1.0,0.978301,1.000000e+00,1.000,0.9999,0.999342,1.000,1.000,0.999996,0.968954,0.998937,0.999986,0.999900,0.999628,0.99982,0.993513,1.000000,0.839682,0.958517,0.854111,0.830532,1.0,1.0,1.0,0.271565,0.352496,0.425447,0.40634,0.183532,0.432515,0.440399,0.423197,0.431632,0.341353,0.406461,0.3439,0.453,0.3435,0.5108,0.2504,0.5628,0.5049,0.4614,0.3437,0.4561,0.34,0.5152,0.2491,0.5628,0.5106,0.4615,0.3252,0.4285,0.3435,0.511,0.2465,0.5639,0.4748,0.4613,0.379106,0.340296,0.413197,0.28876,0.181041,0.475201,0.424837,0.431612,0.431612,0.384973,0.341671,0.414884,0.29817,0.17578,0.471754,0.426679,0.432674,0.432674,0.333396,0.339757,0.396493,0.29529,0.192702,0.481365,0.434049,0.378619,0.426707,0.426707


In [37]:
GetColumnsWithMinAndMaxValuesOutOfRange(df1,0,1)

,MutationAssessor_score,FATHMM_score,PROVEAN_score,MetaSVM_score,CADD_raw_hg19,Eigen-raw_coding,Eigen-PC-raw_coding,GERP++_RS,phyloP100way_vertebrate,phyloP30way_mammalian
min,-4.235,-12.40,-12.58,-2.0058,-2.771251,-2.793459,-2.869872,-12.30,-8.218,-6.245
max,4.925,6.49,13.57,1.3377,7.532029,1.131697,1.048397,6.17,10.003,1.312


In [38]:
GetColumnsWithMinAndMaxValuesOutOfRange(df2,0,1)

,MutationAssessor_score,FATHMM_score,PROVEAN_score,MetaSVM_score,CADD_raw_hg19,Eigen-raw_coding,Eigen-PC-raw_coding,GERP++_RS,phyloP100way_vertebrate,phyloP30way_mammalian
min,-2.58,-12.40,-13.96,-1.2162,-1.016770,-1.83375,-1.913915,-12.20,-3.591,-4.103
max,5.72,5.67,4.15,1.9566,9.391629,1.24247,1.153886,6.17,10.003,1.312


Algunas columnas pueden ser convertidas a valores de patogénico o benigno en función de su valor numérico.
Por tanto, procedo a realizar la creación de una columna numérica que indicará con un 1 si es patógena y un 0 si no lo es según los resultados de estos modelos. Los valores nulos seguirán siendo nulos.

### Creación de las columnas FLAG

In [39]:
def isPatho(x,umbral,sign):
    if x != np.nan and x != None:
        if sign == ">":
            if x > umbral:
                return 1
            else:
                return 0
        else: 
            if sign == ">=":
                if x >= umbral:
                    return 1
                else:
                    return 0
            else: 
                if sign == "<":
                    if x < umbral:
                        return 1
                    else:
                        return 0    
                else: 
                    if sign == "<=":
                        if x <= umbral:
                            return 1
                        else:
                            return 0  
                    else:
                        raise Exception
                
    else:
        return -1

In [40]:
df1['FATHMM_score_FLAG'] = df1['FATHMM_score'].apply(lambda x: isPatho(x,1.5,"<="))
df2['FATHMM_score_FLAG'] = df2['FATHMM_score'].apply(lambda x: isPatho(x,1.5,"<="))

df1['fathmm-MKL_coding_score_FLAG'] = df1['fathmm-MKL_coding_score'].apply(lambda x: isPatho(x,0.5,"<="))
df2['fathmm-MKL_coding_score_FLAG'] = df2['fathmm-MKL_coding_score'].apply(lambda x: isPatho(x,0.5,"<="))

df1['fathmm-XF_coding_score_FLAG'] = df1['fathmm-XF_coding_score'].apply(lambda x: isPatho(x,0.5,"<="))
df2['fathmm-XF_coding_score_FLAG'] = df2['fathmm-XF_coding_score'].apply(lambda x: isPatho(x,0.5,"<="))

df1['BayesDel_addAF_FLAG'] = df1['BayesDel_addAF_score'].apply(lambda x: isPatho(x,0.0692655,"<"))
df2['BayesDel_addAF_FLAG'] = df2['BayesDel_addAF_score'].apply(lambda x: isPatho(x,0.0692655,"<"))
   
df1['BayesDel_noAF_FLAG'] = df1['BayesDel_noAF_score'].apply(lambda x: isPatho(x,-0.0570105,"<"))
df2['BayesDel_noAF_FLAG'] = df2['BayesDel_noAF_score'].apply(lambda x: isPatho(x,-0.0570105,"<"))  

df1['integrated_fitCons_score_FLAG'] = df1['integrated_fitCons_score'].apply(lambda x: isPatho(x,0.7,">"))
df2['integrated_fitCons_score_FLAG'] = df2['integrated_fitCons_score'].apply(lambda x: isPatho(x,0.7,">"))

df1['GM12878_fitCons_score_score_FLAG'] = df1['GM12878_fitCons_score'].apply(lambda x: isPatho(x,0.7,">"))
df2['GM12878_fitCons_score_score_FLAG'] = df2['GM12878_fitCons_score'].apply(lambda x: isPatho(x,0.7,">"))

df1['H1-hESC_fitCons_score_FLAG'] = df1['H1-hESC_fitCons_score'].apply(lambda x: isPatho(x,0.7,">"))
df2['H1-hESC_fitCons_score_FLAG'] = df2['H1-hESC_fitCons_score'].apply(lambda x: isPatho(x,0.7,">"))

df1['HUVEC_fitCons_score_FLAG'] = df1['HUVEC_fitCons_score'].apply(lambda x: isPatho(x,0.7,">"))
df2['HUVEC_fitCons_score_FLAG'] = df2['HUVEC_fitCons_score'].apply(lambda x: isPatho(x,0.7,">"))
    
df1['LRT_score_FLAG'] = df1['LRT_score'].apply(lambda x: isPatho(x, 0.001,"<="))
df2['LRT_score_FLAG'] = df2['LRT_score'].apply(lambda x: isPatho(x, 0.001,"<=")) 
  
df1['MutationAssessor_score_FLAG'] = df1['MutationAssessor_score'].apply(lambda x: isPatho(x, 1.9,">"))
df2['MutationAssessor_score_FLAG'] = df2['MutationAssessor_score'].apply(lambda x: isPatho(x, 1.9,">"))  

df1['MutationTaster_score_FLAG'] = df1['MutationTaster_score'].apply(lambda x: isPatho(x,  0.5,">"))
df2['MutationTaster_score_FLAG'] = df2['MutationTaster_score'].apply(lambda x: isPatho(x,  0.5,">"))

df1['Polyphen2_HDIV_score_FLAG'] = df1['Polyphen2_HDIV_score'].apply(lambda x: isPatho(x,0.453,">="))
df2['Polyphen2_HDIV_score_FLAG'] = df2['Polyphen2_HDIV_score'].apply(lambda x: isPatho(x,0.453,">="))

df1['Polyphen2_HVAR_score_FLAG'] = df1['Polyphen2_HVAR_score'].apply(lambda x: isPatho(x,0.447,">="))
df2['Polyphen2_HVAR_score_FLAG'] = df2['Polyphen2_HVAR_score'].apply(lambda x: isPatho(x,0.447,">="))    
   
df1['PROVEAN_score_FLAG'] = df1['PROVEAN_score'].apply(lambda x: isPatho(x,-2.5,"<="))
df2['PROVEAN_score_FLAG'] = df2['PROVEAN_score'].apply(lambda x: isPatho(x,-2.5,"<="))

df1['SIFT_score_FLAG'] = df1['SIFT_score'].apply(lambda x: isPatho(x, 0.05,"<="))
df2['SIFT_score_FLAG'] = df2['SIFT_score'].apply(lambda x: isPatho(x, 0.05,"<="))
df1['SIFT4G_score_FLAG'] = df1['SIFT4G_score'].apply(lambda x: isPatho(x, 0.05,"<="))
df2['SIFT4G_score_FLAG'] = df2['SIFT4G_score'].apply(lambda x: isPatho(x, 0.05,"<="))    

df1['VEST4_score_FLAG'] = df1['VEST4_score'].apply(lambda x: isPatho(x, 0.05,">="))
df2['VEST4_score_FLAG'] = df2['VEST4_score'].apply(lambda x: isPatho(x, 0.05,">=")) 

df1['GERP++_RS_FLAG'] = df1['GERP++_RS'].apply(lambda x: isPatho(x, 2,">="))
df2['GERP++_RS_FLAG'] = df2['GERP++_RS'].apply(lambda x: isPatho(x, 2,">="))

df1['DEOGEN2_score_FLAG'] = df1['DEOGEN2_score'].apply(lambda x: isPatho(x, 0.5,">="))
df2['DEOGEN2_score_FLAG'] = df2['DEOGEN2_score'].apply(lambda x: isPatho(x, 0.5,">="))

df1['phastCons100way_vertebrate_FLAG'] = df1['phastCons100way_vertebrate'].apply(lambda x: isPatho(x, 0.999,">"))
df2['phastCons100way_vertebrate_FLAG'] = df2['phastCons100way_vertebrate'].apply(lambda x: isPatho(x, 0.999,">"))

df1['phastCons30way_mammalian_FLAG'] = df1['phastCons30way_mammalian'].apply(lambda x: isPatho(x, 0.999,">"))
df2['phastCons30way_mammalian_FLAG'] = df2['phastCons30way_mammalian'].apply(lambda x: isPatho(x, 0.999,">"))

df1['phastCons17way_primate_FLAG'] = df1['phastCons17way_primate'].apply(lambda x: isPatho(x, 0.999,">"))
df2['phastCons17way_primate_FLAG'] = df2['phastCons17way_primate'].apply(lambda x: isPatho(x, 0.999,">")) 

df1['phyloP100way_vertebrate_FLAG'] = df1['phyloP100way_vertebrate'].apply(lambda x: isPatho(x,2,">"))
df2['phyloP100way_vertebrate_FLAG'] = df2['phyloP100way_vertebrate'].apply(lambda x: isPatho(x,2,">"))

df1['phyloP30way_mammalian_FLAG'] = df1['phyloP30way_mammalian'].apply(lambda x: isPatho(x,2,">"))
df2['phyloP30way_mammalian_FLAG'] = df2['phyloP30way_mammalian'].apply(lambda x: isPatho(x,2,">"))

df1['phyloP17way_primate_FLAG'] = df1['phyloP17way_primate'].apply(lambda x: isPatho(x,2,">"))
df2['phyloP17way_primate_FLAG'] = df2['phyloP17way_primate'].apply(lambda x:isPatho(x,2,">"))

df1['SiPhy_29way_logOdds_FLAG'] = df1['SiPhy_29way_logOdds'].apply(lambda x: isPatho(x,12,">="))
df2['SiPhy_29way_logOdds_FLAG'] = df2['SiPhy_29way_logOdds'].apply(lambda x: isPatho(x,12,">="))    

df1['CADD_raw_hg19_FLAG'] = df1['CADD_raw_hg19'].apply(lambda x: isPatho(x,20,">"))
df2['CADD_raw_hg19_FLAG'] = df2['CADD_raw_hg19'].apply(lambda x: isPatho(x,20,">"))
df1['CADD_phred_hg19_FLAG'] = df1['CADD_phred_hg19'].apply(lambda x: isPatho(x,20,">"))
df2['CADD_phred_hg19_FLAG'] = df2['CADD_phred_hg19'].apply(lambda x: isPatho(x,20,">"))

df1['DANN_score_FLAG'] = df1['DANN_score'].apply(lambda x: isPatho(x,0.99,">="))
df2['DANN_score_FLAG'] = df2['DANN_score'].apply(lambda x: isPatho(x,0.99,">="))   

df1['GenoCanyon_score_FLAG'] = df1['GenoCanyon_score'].apply(lambda x: isPatho(x,0.999,">"))
df2['GenoCanyon_score_FLAG'] = df2['GenoCanyon_score'].apply(lambda x: isPatho(x,0.999,">"))
    
df1['MetaLR_score_FLAG'] = df1['MetaLR_score'].apply(lambda x: isPatho(x,0.5,">"))
df2['MetaLR_score_FLAG'] = df2['MetaLR_score'].apply(lambda x: isPatho(x,0.5,">"))    

df1['M-CAP_score_FLAG'] = df1['M-CAP_score'].apply(lambda x: isPatho(x,0.025,">"))
df2['M-CAP_score_FLAG'] = df2['M-CAP_score'].apply(lambda x: isPatho(x,0.025,">"))

df1['MetaSVM_score_FLAG'] = df1['MetaSVM_score'].apply(lambda x: isPatho(x,0,">"))
df2['MetaSVM_score_FLAG'] = df2['MetaSVM_score'].apply(lambda x: isPatho(x,0,">"))

df1['LIST-S2_score_FLAG'] = df1['LIST-S2_score'].apply(lambda x: isPatho(x,0.85,">="))
df2['LIST-S2_score_FLAG'] = df2['LIST-S2_score'].apply(lambda x: isPatho(x,0.85,">="))

Para algunas columnas, como vimos antes, podemos invertir sus valores; es decir, si su escala trabaja desde -10 a 10, los valores más altos pasarán a ser los más bajos y a la inversa.

Para ello, usaremos la ecuación de una recta entre dos puntos: 
Sea C una columna de datos con valores mínimos xMin, y máximo xMax, a la cúal queremos cambiar de escala a los valores definidos entre xMin' y xMax'.

Definimos a la recta A como la recta definida por los valores mínimo xMin y máximo xMax de una columna C para los valores X de sus puntos, y por los valores mínimo xMin' y máximo xMax' de los valores a los que queremos convertirlos, para los valores Y de sus puntos. De tal forma que, cada punto indica con su coordenada X el valor que toma en la escala inicial, y con su coordenada Y su valor en la nueva escala.
De esta forma, podemos utilizar la ecuación de una recta entre dos puntos para hallar el valor de dicha Y a partir de un valor X, sabiendo que dos puntos de la recta son (xMin, xMin') y (xMax, xMax').

In [41]:
def equationBetweenTwoPoints(x1, x2, y1, y2, x):
    if (x2 - x1 != 0):
        return (((x - x1)/(x2-x1))*(y2-y1)) + y1

def reverseScale(x, xMin, xMax):
    return equationBetweenTwoPoints(xMin, xMax, xMax, xMin, x)

def normalizeValues(x, xMin, xMax):
    return equationBetweenTwoPoints(xMin, xMax, 0, 1, x)

### Normalización de los rangos en los datasets

In [42]:
min_f_1, max_f_1 = df1['FATHMM_score'].describe()[['min','max']].tolist()
min_p_1, max_p_1 = df1['PROVEAN_score'].describe()[['min','max']].tolist()
min_s_1, max_s_1 = df1['SIFT_score'].describe()[['min','max']].tolist()
min_s4_1, max_s4_1 = df1['SIFT4G_score'].describe()[['min','max']].tolist()
min_lrt_1, max_lrt_1 = df1['LRT_score'].describe()[['min','max']].tolist()

df1['FATHMM_score_reverted'] = df1['FATHMM_score'].apply(lambda x: reverseScale(x, min_f_1, max_f_1))
df1['PROVEAN_score_reverted'] = df1['PROVEAN_score'].apply(lambda x: reverseScale(x, min_p_1, max_p_1))
df1['SIFT_score_reverted'] = df1['SIFT_score'].apply(lambda x: reverseScale(x, min_s_1, max_s_1))
df1['SIFT4G_score_reverted'] = df1['SIFT4G_score'].apply(lambda x: reverseScale(x, min_s4_1, max_s4_1))
df1['LRT_score_reverted'] = df1['LRT_score'].apply(lambda x: reverseScale(x, min_lrt_1, max_lrt_1))


In [43]:
min_f_2, max_f_2 = df2['FATHMM_score'].describe()[['min','max']].tolist()
min_p_2, max_p_2 = df2['PROVEAN_score'].describe()[['min','max']].tolist()
min_s_2, max_s_2 = df2['SIFT_score'].describe()[['min','max']].tolist()
min_s4_2, max_s4_2 = df2['SIFT4G_score'].describe()[['min','max']].tolist()
min_lrt_2, max_lrt_2 = df1['LRT_score'].describe()[['min','max']].tolist()

df2['FATHMM_score_reverted'] = df2['FATHMM_score'].apply(lambda x: reverseScale(x, min_f_2, max_f_2))
df2['PROVEAN_score_reverted'] = df2['PROVEAN_score'].apply(lambda x: reverseScale(x, min_p_2, max_p_2))
df2['SIFT_score_reverted'] = df2['SIFT_score'].apply(lambda x: reverseScale(x, min_s_2, max_s_2))
df2['SIFT4G_score_reverted'] = df2['SIFT4G_score'].apply(lambda x: reverseScale(x, min_s4_2, max_s4_2))
df2['LRT_score_reverted'] = df2['LRT_score'].apply(lambda x: reverseScale(x, min_lrt_2, max_lrt_2))

In [44]:
### PROVEAN score = -14 to 14.

df1['PROVEAN_score_normalized'] = df1['PROVEAN_score_reverted'].apply(lambda x: normalizeValues(x, -14, 14))
df2['PROVEAN_score_normalized'] = df2['PROVEAN_score_reverted'].apply(lambda x: normalizeValues(x, -14, 14))

In [45]:
### FATHMM_score = -16.13 to 10.64

df1['FATHMM_score_normalized'] = df1['FATHMM_score_reverted'].apply(lambda x: normalizeValues(x, -16.13, 10.64))
df2['FATHMM_score_normalized'] = df2['FATHMM_score_reverted'].apply(lambda x: normalizeValues(x, -16.13, 10.64))

In [46]:
### SIFT_score  = 0 to 1.

df1['SIFT_score_normalized'] = df1['SIFT_score_reverted'].apply(lambda x: normalizeValues(x, 0, 1))
df2['SIFT_score_normalized'] = df2['SIFT_score_reverted'].apply(lambda x: normalizeValues(x, 0, 1))

In [47]:
### SIFT_score 4G =  0 to 1.

df1['SIFT4G_score_normalized'] = df1['SIFT4G_score_reverted'].apply(lambda x: normalizeValues(x, 0, 1))
df2['SIFT4G_score_normalized'] = df2['SIFT4G_score_reverted'].apply(lambda x: normalizeValues(x, 0, 1))

In [48]:
del df1['FATHMM_score_reverted']
del df2['FATHMM_score_reverted']
del df1['PROVEAN_score_reverted']
del df2['PROVEAN_score_reverted']
del df1['SIFT_score_reverted']
del df2['SIFT_score_reverted']
del df1['SIFT4G_score_reverted']
del df2['SIFT4G_score_reverted']

In [49]:
### MutationAssessor_score = -5.2 to 6.5.

df1['MutationAssessor_score_normalized'] = df1['MutationAssessor_score'].apply(lambda x: normalizeValues(x, -5.2, 6.5))
df2['MutationAssessor_score_normalized'] = df2['MutationAssessor_score'].apply(lambda x: normalizeValues(x, -5.2, 6.5))

In [50]:
### LRT_Omega: 0 to 7780.54

df1['LRT_Omega_normalized'] = df1['LRT_Omega'].apply(lambda x: normalizeValues(x, 0, 7780.54))
df2['LRT_Omega_normalized'] = df2['LRT_Omega'].apply(lambda x: normalizeValues(x, 0, 7780.54))

In [51]:
### MetaSVM_score: -2 to 3.

df1['MetaSVM_score_normalized'] = df1['MetaSVM_score'].apply(lambda x: normalizeValues(x, -2, 3))
df2['MetaSVM_score_normalized'] = df2['MetaSVM_score'].apply(lambda x: normalizeValues(x, -2, 3))

In [52]:
### MPC_score: 0 to 5.

df1['MPC_score_normalized'] = df1['MPC_score'].apply(lambda x: normalizeValues(x, 0, 5))
df2['MPC_score_normalized'] = df2['MPC_score'].apply(lambda x: normalizeValues(x, 0, 5))

In [53]:
### GERP++_RS:  -12.3 to 6.17.

df1['GERP++_RS_normalized'] = df1['GERP++_RS'].apply(lambda x: normalizeValues(x, -12.3, 6.17))
df2['GERP++_RS_normalized'] = df2['GERP++_RS'].apply(lambda x: normalizeValues(x, -12.3, 6.17))

In [54]:
### phyloP100way_vertebrate: -20.0 to 10.003 

df1['phyloP100way_vertebrate_normalized'] = df1['phyloP100way_vertebrate'].apply(lambda x: normalizeValues(x, -20.0, 10.003))
df2['phyloP100way_vertebrate_normalized'] = df2['phyloP100way_vertebrate'].apply(lambda x: normalizeValues(x, -20.0, 10.003))

In [55]:
### phyloP30way_mammalian: -20 to 1.312

df1['phyloP30way_mammalian_normalized'] = df1['phyloP30way_mammalian'].apply(lambda x: normalizeValues(x, -20.0, 1.312))
df2['phyloP30way_mammalian_normalized'] = df2['phyloP30way_mammalian'].apply(lambda x: normalizeValues(x, -20.0, 1.312))

In [56]:
### phyloP17way_primate: -13.362 to 0.756

df1['phyloP17way_primate_normalized'] = df1['phyloP17way_primate'].apply(lambda x: normalizeValues(x, -13.362, 0.756))
df2['phyloP17way_primate_normalized'] = df2['phyloP17way_primate'].apply(lambda x: normalizeValues(x, -13.362, 0.756))

In [57]:
### SiPhy_29way_logOdds: 0 to 37.9718

df1['SiPhy_29way_logOdds_normalized'] = df1['SiPhy_29way_logOdds'].apply(lambda x: normalizeValues(x, 0, 37.9718))
df2['SiPhy_29way_logOdds_normalized'] = df2['SiPhy_29way_logOdds'].apply(lambda x: normalizeValues(x, 0, 37.9718))

In [58]:
### bStatistic: 0 to 1000

df1['bStatistic_normalized'] = df1['bStatistic'].apply(lambda x: normalizeValues(x, 0, 1000))
df2['bStatistic_normalized'] = df2['bStatistic'].apply(lambda x: normalizeValues(x, 0, 1000))

In [59]:
### BayesDel_addAF: -1.11707 to 0.750927

df1['BayesDel_addAF_score_normalized'] = df1['BayesDel_addAF_score'].apply(lambda x: normalizeValues(x, -1.11707, 0.750927))
df2['BayesDel_addAF_score_normalized'] = df2['BayesDel_addAF_score'].apply(lambda x: normalizeValues(x, -1.11707, 0.750927))

In [60]:
### BayesDel_noAF_score: -1.31914 to 0.840878.

df1['BayesDel_noAF_score_normalized'] = df1['BayesDel_noAF_score'].apply(lambda x: normalizeValues(x, -1.31914, 0.840878))
df2['BayesDel_noAF_score_normalized'] = df2['BayesDel_noAF_score'].apply(lambda x: normalizeValues(x, -1.31914, 0.840878))

In [61]:
GetColumnsWithMinAndMaxValuesInRange(df1,0,1)

,SIFT_score,SIFT4G_score,Polyphen2_HDIV_score,Polyphen2_HVAR_score,LRT_score,MutationTaster_score,VEST4_score,MetaLR_score,M-CAP_score,REVEL_score,MutPred_score,MVP_score,PrimateAI_score,DEOGEN2_score,ClinPred_score,LIST-S2_score,DANN_score,fathmm-MKL_coding_score,fathmm-XF_coding_score,GenoCanyon_score,integrated_fitCons_score,GM12878_fitCons_score,H1-hESC_fitCons_score,HUVEC_fitCons_score,phastCons100way_vertebrate,phastCons30way_mammalian,phastCons17way_primate,1000Gp3_AF,1000Gp3_AFR_AF,1000Gp3_EUR_AF,1000Gp3_AMR_AF,1000Gp3_EAS_AF,1000Gp3_SAS_AF,TWINSUK_AF,ALSPAC_AF,UK10K_AF,ESP6500_AA_AF,ESP6500_EA_AF,ExAC_AF,ExAC_Adj_AF,ExAC_AFR_AF,ExAC_AMR_AF,ExAC_EAS_AF,ExAC_FIN_AF,ExAC_NFE_AF,ExAC_SAS_AF,ExAC_nonTCGA_AF,ExAC_nonTCGA_Adj_AF,ExAC_nonTCGA_AFR_AF,ExAC_nonTCGA_AMR_AF,ExAC_nonTCGA_EAS_AF,ExAC_nonTCGA_FIN_AF,ExAC_nonTCGA_NFE_AF,ExAC_nonTCGA_SAS_AF,ExAC_nonpsych_AF,ExAC_nonpsych_Adj_AF,ExAC_nonpsych_AFR_AF,ExAC_nonpsych_AMR_AF,ExAC_nonpsych_EAS_AF,ExAC_nonpsych_FIN_AF,ExAC_nonpsych_NFE_AF,ExAC_nonpsych_SAS_AF,gnomAD_exomes_AF,gnomAD_exomes_AFR_AF,gnomAD_exomes_AMR_AF,gnomAD_exomes_ASJ_AF,gnomAD_exomes_EAS_AF,gnomAD_exomes_FIN_AF,gnomAD_exomes_NFE_AF,gnomAD_exomes_SAS_AF,gnomAD_exomes_POPMAX_AF,gnomAD_exomes_controls_AF,gnomAD_exomes_controls_AFR_AF,gnomAD_exomes_controls_AMR_AF,gnomAD_exomes_controls_ASJ_AF,gnomAD_exomes_controls_EAS_AF,gnomAD_exomes_controls_FIN_AF,gnomAD_exomes_controls_NFE_AF,gnomAD_exomes_controls_SAS_AF,gnomAD_exomes_controls_POPMAX_AF,gnomAD_genomes_AF,gnomAD_genomes_AFR_AF,gnomAD_genomes_AMR_AF,gnomAD_genomes_ASJ_AF,gnomAD_genomes_EAS_AF,gnomAD_genomes_FIN_AF,gnomAD_genomes_NFE_AF,gnomAD_genomes_AMI_AF,gnomAD_genomes_SAS_AF,gnomAD_genomes_POPMAX_AF,FATHMM_score_FLAG,fathmm-MKL_coding_score_FLAG,fathmm-XF_coding_score_FLAG,BayesDel_addAF_FLAG,BayesDel_noAF_FLAG,integrated_fitCons_score_FLAG,GM12878_fitCons_score_score_FLAG,H1-hESC_fitCons_score_FLAG,HUVEC_fitCons_score_FLAG,LRT_score_FLAG,MutationAssessor_score_FLAG,MutationTaster_score_FLAG,Polyphen2_HDIV_score_FLAG,Polyphen2_HVAR_score_FLAG,PROVEAN_score_FLAG,SIFT_score_FLAG,SIFT4G_score_FLAG,VEST4_score_FLAG,GERP++_RS_FLAG,DEOGEN2_score_FLAG,phastCons100way_vertebrate_FLAG,phastCons30way_mammalian_FLAG,phastCons17way_primate_FLAG,phyloP100way_vertebrate_FLAG,phyloP30way_mammalian_FLAG,phyloP17way_primate_FLAG,SiPhy_29way_logOdds_FLAG,CADD_raw_hg19_FLAG,CADD_phred_hg19_FLAG,DANN_score_FLAG,GenoCanyon_score_FLAG,MetaLR_score_FLAG,M-CAP_score_FLAG,MetaSVM_score_FLAG,LIST-S2_score_FLAG,LRT_score_reverted,PROVEAN_score_normalized,FATHMM_score_normalized,SIFT_score_normalized,SIFT4G_score_normalized,MutationAssessor_score_normalized,LRT_Omega_normalized,MPC_score_normalized,GERP++_RS_normalized,phyloP100way_vertebrate_normalized,phyloP30way_mammalian_normalized,phyloP17way_primate_normalized,SiPhy_29way_logOdds_normalized,bStatistic_normalized,BayesDel_addAF_score_normalized,BayesDel_noAF_score_normalized
min,0.0,0.0,0.0,0.0,0.0,1.000000e-37,0.002,0.0000,0.000950,0.000,0.049,0.013882,0.154256,0.000139,0.000010,0.002400,0.036852,0.00000,0.003400,0.000001,0.001892,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050714,0.139335,0.0,0.0,0.082479,0.000000,0.000076,0.0,0.392694,0.645411,0.478042,0.000058,0.0,0.017912,0.056092
max,1.0,1.0,1.0,1.0,1.0,1.000000e+00,0.997,0.9999,0.994657,0.992,1.000,1.000000,0.973357,0.988043,0.999961,0.999865,0.999630,0.99907,0.987401,1.000000,0.839682,0.958517,0.858454,0.836244,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.

In [62]:
GetColumnsWithMinAndMaxValuesInRange(df1,0,1)

,SIFT_score,SIFT4G_score,Polyphen2_HDIV_score,Polyphen2_HVAR_score,LRT_score,MutationTaster_score,VEST4_score,MetaLR_score,M-CAP_score,REVEL_score,MutPred_score,MVP_score,PrimateAI_score,DEOGEN2_score,ClinPred_score,LIST-S2_score,DANN_score,fathmm-MKL_coding_score,fathmm-XF_coding_score,GenoCanyon_score,integrated_fitCons_score,GM12878_fitCons_score,H1-hESC_fitCons_score,HUVEC_fitCons_score,phastCons100way_vertebrate,phastCons30way_mammalian,phastCons17way_primate,1000Gp3_AF,1000Gp3_AFR_AF,1000Gp3_EUR_AF,1000Gp3_AMR_AF,1000Gp3_EAS_AF,1000Gp3_SAS_AF,TWINSUK_AF,ALSPAC_AF,UK10K_AF,ESP6500_AA_AF,ESP6500_EA_AF,ExAC_AF,ExAC_Adj_AF,ExAC_AFR_AF,ExAC_AMR_AF,ExAC_EAS_AF,ExAC_FIN_AF,ExAC_NFE_AF,ExAC_SAS_AF,ExAC_nonTCGA_AF,ExAC_nonTCGA_Adj_AF,ExAC_nonTCGA_AFR_AF,ExAC_nonTCGA_AMR_AF,ExAC_nonTCGA_EAS_AF,ExAC_nonTCGA_FIN_AF,ExAC_nonTCGA_NFE_AF,ExAC_nonTCGA_SAS_AF,ExAC_nonpsych_AF,ExAC_nonpsych_Adj_AF,ExAC_nonpsych_AFR_AF,ExAC_nonpsych_AMR_AF,ExAC_nonpsych_EAS_AF,ExAC_nonpsych_FIN_AF,ExAC_nonpsych_NFE_AF,ExAC_nonpsych_SAS_AF,gnomAD_exomes_AF,gnomAD_exomes_AFR_AF,gnomAD_exomes_AMR_AF,gnomAD_exomes_ASJ_AF,gnomAD_exomes_EAS_AF,gnomAD_exomes_FIN_AF,gnomAD_exomes_NFE_AF,gnomAD_exomes_SAS_AF,gnomAD_exomes_POPMAX_AF,gnomAD_exomes_controls_AF,gnomAD_exomes_controls_AFR_AF,gnomAD_exomes_controls_AMR_AF,gnomAD_exomes_controls_ASJ_AF,gnomAD_exomes_controls_EAS_AF,gnomAD_exomes_controls_FIN_AF,gnomAD_exomes_controls_NFE_AF,gnomAD_exomes_controls_SAS_AF,gnomAD_exomes_controls_POPMAX_AF,gnomAD_genomes_AF,gnomAD_genomes_AFR_AF,gnomAD_genomes_AMR_AF,gnomAD_genomes_ASJ_AF,gnomAD_genomes_EAS_AF,gnomAD_genomes_FIN_AF,gnomAD_genomes_NFE_AF,gnomAD_genomes_AMI_AF,gnomAD_genomes_SAS_AF,gnomAD_genomes_POPMAX_AF,FATHMM_score_FLAG,fathmm-MKL_coding_score_FLAG,fathmm-XF_coding_score_FLAG,BayesDel_addAF_FLAG,BayesDel_noAF_FLAG,integrated_fitCons_score_FLAG,GM12878_fitCons_score_score_FLAG,H1-hESC_fitCons_score_FLAG,HUVEC_fitCons_score_FLAG,LRT_score_FLAG,MutationAssessor_score_FLAG,MutationTaster_score_FLAG,Polyphen2_HDIV_score_FLAG,Polyphen2_HVAR_score_FLAG,PROVEAN_score_FLAG,SIFT_score_FLAG,SIFT4G_score_FLAG,VEST4_score_FLAG,GERP++_RS_FLAG,DEOGEN2_score_FLAG,phastCons100way_vertebrate_FLAG,phastCons30way_mammalian_FLAG,phastCons17way_primate_FLAG,phyloP100way_vertebrate_FLAG,phyloP30way_mammalian_FLAG,phyloP17way_primate_FLAG,SiPhy_29way_logOdds_FLAG,CADD_raw_hg19_FLAG,CADD_phred_hg19_FLAG,DANN_score_FLAG,GenoCanyon_score_FLAG,MetaLR_score_FLAG,M-CAP_score_FLAG,MetaSVM_score_FLAG,LIST-S2_score_FLAG,LRT_score_reverted,PROVEAN_score_normalized,FATHMM_score_normalized,SIFT_score_normalized,SIFT4G_score_normalized,MutationAssessor_score_normalized,LRT_Omega_normalized,MPC_score_normalized,GERP++_RS_normalized,phyloP100way_vertebrate_normalized,phyloP30way_mammalian_normalized,phyloP17way_primate_normalized,SiPhy_29way_logOdds_normalized,bStatistic_normalized,BayesDel_addAF_score_normalized,BayesDel_noAF_score_normalized
min,0.0,0.0,0.0,0.0,0.0,1.000000e-37,0.002,0.0000,0.000950,0.000,0.049,0.013882,0.154256,0.000139,0.000010,0.002400,0.036852,0.00000,0.003400,0.000001,0.001892,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050714,0.139335,0.0,0.0,0.082479,0.000000,0.000076,0.0,0.392694,0.645411,0.478042,0.000058,0.0,0.017912,0.056092
max,1.0,1.0,1.0,1.0,1.0,1.000000e+00,0.997,0.9999,0.994657,0.992,1.000,1.000000,0.973357,0.988043,0.999961,0.999865,0.999630,0.99907,0.987401,1.000000,0.839682,0.958517,0.858454,0.836244,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.

In [63]:
GetColumnsWithMinAndMaxValuesOutOfRange(df1,0,1)

,MutationAssessor_score,FATHMM_score,PROVEAN_score,MetaSVM_score,CADD_raw_hg19,Eigen-raw_coding,Eigen-PC-raw_coding,GERP++_RS,phyloP100way_vertebrate,phyloP30way_mammalian
min,-4.235,-12.40,-12.58,-2.0058,-2.771251,-2.793459,-2.869872,-12.30,-8.218,-6.245
max,4.925,6.49,13.57,1.3377,7.532029,1.131697,1.048397,6.17,10.003,1.312


In [64]:
GetColumnsWithMinAndMaxValuesOutOfRange(df2,0,1)

,MutationAssessor_score,FATHMM_score,PROVEAN_score,MetaSVM_score,CADD_raw_hg19,Eigen-raw_coding,Eigen-PC-raw_coding,GERP++_RS,phyloP100way_vertebrate,phyloP30way_mammalian
min,-2.58,-12.40,-13.96,-1.2162,-1.016770,-1.83375,-1.913915,-12.20,-3.591,-4.103
max,5.72,5.67,4.15,1.9566,9.391629,1.24247,1.153886,6.17,10.003,1.312


In [65]:
df1.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22631 entries, 0 to 10895
Data columns (total 171 columns):
 #   Column                              Dtype  
---  ------                              -----  
 0   #chr                                object 
 1   pos(1-based)                        int64  
 2   ref                                 object 
 3   alt                                 object 
 4   aaref                               object 
 5   aaalt                               object 
 6   hg19_chr                            object 
 7   hg19_pos(1-based)                   int64  
 8   aapos                               object 
 9   SIFT_score                          float64
 10  SIFT4G_score                        float64
 11  Polyphen2_HDIV_score                float64
 12  Polyphen2_HVAR_score                float64
 13  LRT_score                           float64
 14  LRT_Omega                           float64
 15  MutationTaster_score                float64
 16  Mut

In [66]:
df2.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12167 entries, 0 to 5514
Data columns (total 171 columns):
 #   Column                              Dtype  
---  ------                              -----  
 0   #chr                                object 
 1   pos(1-based)                        int64  
 2   ref                                 object 
 3   alt                                 object 
 4   aaref                               object 
 5   aaalt                               object 
 6   hg19_chr                            object 
 7   hg19_pos(1-based)                   int64  
 8   aapos                               object 
 9   SIFT_score                          float64
 10  SIFT4G_score                        float64
 11  Polyphen2_HDIV_score                float64
 12  Polyphen2_HVAR_score                float64
 13  LRT_score                           float64
 14  LRT_Omega                           float64
 15  MutationTaster_score                float64
 16  Muta

## Exportamos los Datasets en formato CSV para su posterior uso

In [67]:
df1.to_csv(path_or_buf="datasets/clean_bening.csv")
df2.to_csv(path_or_buf="datasets/clean_patho.csv")